## Data

In oreder to make our analysis we need to obtain some data:
- We need to construct a mesh of areas covering the center of Aarhus, Denmark
- From these locations we obtain Forusquare venue data of food venues and seafood restaurant venues, and visualize them

### We find the center of Aarhus (Arhus Cathedral)

First we import packages to handle folium.

In [3]:
import requests
#!pip install geopy

from geopy.geocoders import Nominatim
geolocator = Nominatim()
adresse = 'Aarhus Domkirke'
try:
    location = geolocator.geocode(adresse)
    print('Arhus Centrum is in ', location.latitude, location.longitude)
except:
    print(adresse)
aarhus_centrum = [location.latitude, location.longitude]
print(aarhus_centrum)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


Arhus Centrum is in  56.15685435 10.2106463239974
[56.15685435, 10.2106463239974]


### We make a mesh of neighbourhoods

#### First we convert from latitude/longitude to cartesian measures

In [9]:
#!pip install shapely
import shapely.geometry

#!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Aarhus Centrum longitude={}, latitude={}'.format(aarhus_centrum[1], aarhus_centrum[0]))
x, y = lonlat_to_xy(aarhus_centrum[1], aarhus_centrum[0])
print('Aarhus centrum UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Aarhus centrum longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Aarhus Centrum longitude=10.2106463239974, latitude=56.15685435
Aarhus centrum UTM X=202641.0637672961, Y=6233869.4107194785
Aarhus centrum longitude=10.210646323997398, latitude=56.15685435000001


#### Then we construct the neighbourhood centers

In [5]:
aarhus_centrum_x, aarhus_centrum_y = lonlat_to_xy(aarhus_centrum[1], aarhus_centrum[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = aarhus_centrum_x - 1800  # we start 1.800 metres from the center
x_step = 300
y_min = aarhus_centrum_y - 1800 - (int(22/k)*k*300 - 3600)/2
y_step = 300 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(22/k)):
    y = y_min + i * y_step
    x_offset = 150 if i%2==0 else 0
    for j in range(0, 10):                      # the steps stop at 10 because wa have seawater to the east
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(aarhus_centrum_x, aarhus_centrum_y, x, y)
        if (distance_from_center <= 1801):      # only append points within the 1.800 meter radius
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

116 candidate neighborhood centers generated.


#### The neighbourhood centers are plotted

In [7]:
#!pip install folium
import folium

In [8]:
map_aarhus = folium.Map(location=aarhus_centrum, zoom_start=13)
folium.Marker(aarhus_centrum, popup='Aarhus Domkirke').add_to(map_aarhus)
for lat, lon in zip(latitudes, longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_aarhus) 
map_aarhus

#### The locations are imported to a dataframe

In [10]:
import pandas as pd

df_locations = pd.DataFrame({'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys})

df_locations.head()

,Latitude,Longitude,X,Y
0,56.141454,10.205314,202191.063767,6.232181e+06
1,56.141641,10.210125,202491.063767,6.232181e+06
2,56.141828,10.214936,202791.063767,6.232181e+06
3,56.142014,10.219747,203091.063767,6.232181e+06
4,56.143499,10.197807,201741.063767,6.232440e+06


### Foursquare venue information is collected

#### We set up to import Foursquare venue information

In [15]:
CLIENT_ID = '1APIKEUUY3N0O0CG22IPYPWFVL33NLZPJN4RVSHXBO315QIS' # your Foursquare ID
CLIENT_SECRET = 'Y21WCG50Q5S4VLNYQOFIUQ1IHBSQTDUWLDS0NVQX1QBXIDQU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

food_category = '4d4b7105d754a06374d81259'
seafood_category = '4bf58dd8d48988d1ce941735'

#!pip install foursquare
import foursquare
from pandas.io.json import json_normalize

Your credentails:
CLIENT_ID: 1APIKEUUY3N0O0CG22IPYPWFVL33NLZPJN4RVSHXBO315QIS
CLIENT_SECRET:Y21WCG50Q5S4VLNYQOFIUQ1IHBSQTDUWLDS0NVQX1QBXIDQU


#### We make a function to call the Foursquare venue information

In [16]:
def getNearbyVenues(latitudes, longitudes, category, radius=350, LIMIT=100):  #radius = 350 to get an overlap
    venues_list=[]
    for lat, lng in zip(latitudes, longitudes):
        #print(lat)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            category,
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### We find all food related venues

In [17]:
food_venues = getNearbyVenues(latitudes, longitudes, food_category)
food_venues.head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,56.141454,10.205314,Marselis Pizzeriaer Og Sandwich,56.141987,10.204273,Pizza Place
1,56.141454,10.205314,Marselis Pizzeria,56.143119,10.205162,Pizza Place
2,56.141454,10.205314,Aspendos,56.143428,10.205480,Pizza Place
3,56.141454,10.205314,Café Skovbrynet,56.139254,10.206010,Café
4,56.141641,10.210125,Marselis Pizzeria,56.143119,10.205162,Pizza Place


In [18]:
seafood_venues = getNearbyVenues(latitudes, longitudes, seafood_category)
seafood_venues.head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,56.141828,10.214936,Restaurant Seafood,56.138691,10.215075,Seafood Restaurant
1,56.155598,10.208385,Sushimania,56.158264,10.207182,Seafood Restaurant
2,56.155785,10.213198,Restaurant Pihlkjær,56.158595,10.212578,Seafood Restaurant
3,56.157830,10.205688,Gambas,56.158891,10.206710,Seafood Restaurant
4,56.157830,10.205688,Sushimania,56.158264,10.207182,Seafood Restaurant


### Food venues and seafood restaurants are plotted (seafood restaurants are red)

In [21]:
map_aarhus = folium.Map(location=aarhus_centrum, zoom_start=14)
folium.Marker(aarhus_centrum, popup='Aarhus Domkirke').add_to(map_aarhus)
for res in food_venues.values:
    lat = res[3]; lon = res[4]
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_aarhus) 
for res in seafood_venues.values:
    lat = res[3]; lon = res[4]
    folium.CircleMarker([lat, lon], radius=2, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_aarhus)   
map_aarhus

#### And the same map with the areas

In [20]:
map_aarhus = folium.Map(location=aarhus_centrum, zoom_start=14)
folium.Marker(aarhus_centrum, popup='Aarhus Domkirke').add_to(map_aarhus)
for res in food_venues.values:
    lat = res[3]; lon = res[4]
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_aarhus) 
for res in seafood_venues.values:
    lat = res[3]; lon = res[4]
    folium.CircleMarker([lat, lon], radius=2, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_aarhus)   
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=150, color='grey', fill=False).add_to(map_aarhus)
map_aarhus

### At end we find all venues in each area

In [25]:
def getAllNearbyVenues(latitudes, longitudes, radius=350, LIMIT=100):  #radius = 350 to get an overlap
    venues_list=[]
    for lat, lng in zip(latitudes, longitudes):
        #print(lat)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

all_venues = getAllNearbyVenues(latitudes, longitudes)
all_venues.head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,56.141454,10.205314,Jazz Og Folk Teltet,56.143377,10.205413,Jazz Club
1,56.141454,10.205314,Bus 100 (Odder - Århus - Hornslet / Hornslet -...,56.143399,10.205403,Bus Line
2,56.141454,10.205314,Filtenborgs Plads,56.139172,10.206398,Plaza
3,56.141454,10.205314,Kiosken v. Tangkrogen,56.138999,10.206916,Candy Store
4,56.141454,10.205314,Århus Kræmmermarked,56.139000,10.208425,Shop & Service


These venues we will use to make a k-means cluster analysis of all the neighbourhoods

## Analysis

 - Make a k-means analysis. Analyse the clusters, in order to find a suitable cluster to locate a new seafood restaurant. 
 - Find a spot away fram other seafood restaurants.
 - Find a spot not crowded with food venues.

## Conclusion and recommendations

Find most suitable place for a seafood restaurant.